
### Requisitos para os arquivos .csv

Objetivo: realizar processo de ETL e disponibilizar os dados em camada adequada.

1. Ler o arquivo csv do Datalake [ok]
2. Renomear colunas para português usando snake case [ok]
3. Definir a tipagem de colunas correta [ok]
4. Coluna com data de cosumo do arquivo [ok]
5. Salvar dados em formato parquet na camada adequada [ok]

In [11]:
%run ../../Configuração-e-Utilitários/Variaveis.ipynb

In [12]:
%run ../../Configuração-e-Utilitários/Funções.ipynb

### Obtendo valor do parâmetro do notebook

In [15]:
dbutils.widgets.text('nome_arquivo_fonte', '')
arquivo_fonte = dbutils.widgets.get('nome_arquivo_fonte')


### Lendo os dados do arquivo

In [18]:
df = (spark.read
               .option("encoding", "UTF-16")
               .option("header", "true") # Detecção do cabeçalho
               .option("inferSchema", "true") # Detecção do tipo de dados
               .csv(f"{caminho_bronze}/{arquivo_fonte}.csv") # Caminho do arquivo
)
df.show()

+-----+--------------------+--------------------+--------------------+-------+--------+-------+--------------------+
|   id|                name|             address|                city|country|capacity|surface|               image|
+-----+--------------------+--------------------+--------------------+-------+--------+-------+--------------------+
|  204|Estadio Jornalist...|Rua Professor Eur...|Rio de Janeiro, R...| Brazil|   78838|  grass|https://media-2.a...|
|  216|    Arena Fonte Nova|Rua Lions Club, N...|     Salvador, Bahia| Brazil|   56500|  grass|https://media-1.a...|
|  244|Estádio José Pinh...|Avenida Padre Cac...|Porto Alegre, Rio...| Brazil|   50128|  grass|https://media-3.a...|
|  218|Estádio Nilton Sa...|Rua Arquias Corde...|      Rio de Janeiro| Brazil|   46931|  grass|https://media-1.a...|
|  258|      Allianz Parque|Rua Turiaçu 1840,...|São Paulo, São Paulo| Brazil|   43713|  grass|https://media-2.a...|
|  259|Estádio Durival d...|Rua Engenheiro Re...|    Curitiba, P

In [19]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- capacity: integer (nullable = true)
 |-- surface: string (nullable = true)
 |-- image: string (nullable = true)



### Renomeando colunas para português e alterando valores

In [20]:
if arquivo_fonte == 'estadios':
    df = df \
        .withColumnRenamed("id", "id_estadio") \
        .withColumnRenamed("name", "nm_estadio") \
        .withColumnRenamed("address", "endereço") \
        .withColumnRenamed("city", "cidade") \
        .withColumnRenamed("country", "pais") \
        .withColumnRenamed("capacity", "capacidade") \
        .withColumnRenamed("surface", "superficie") \
        .withColumnRenamed("image", "link_imagem")

elif arquivo_fonte == 'torneios':

    from pyspark.sql.functions import when, substring

    df = df \
        .withColumnRenamed("league_id", "id_torneio") \
        .withColumnRenamed("league_name", "nome_torneio") \
        .withColumnRenamed("country_name", "pais") \
        .withColumnRenamed("country_code", "sigla_do_pais") \
        .withColumnRenamed("season_year", "ano_da_temporada") \
        .withColumnRenamed("season_start", "dt_inicio_da_temporada") \
        .withColumnRenamed("season_end", "dt_fim_da_temporada") \
        .withColumnRenamed("season_current", "temporada_atual")
    
    df = df.withColumn('temporada_atual', df.temporada_atual.cast('string'))
    df = df.withColumn('temporada_atual', when(substring(df.temporada_atual, 1, 1) == 'T', 'Sim').otherwise('Não'))

df.show()

+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+
|id_estadio|          nm_estadio|            endereço|              cidade|  pais|capacidade|superficie|         link_imagem|
+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+
|       204|Estadio Jornalist...|Rua Professor Eur...|Rio de Janeiro, R...|Brazil|     78838|     grass|https://media-2.a...|
|       216|    Arena Fonte Nova|Rua Lions Club, N...|     Salvador, Bahia|Brazil|     56500|     grass|https://media-1.a...|
|       244|Estádio José Pinh...|Avenida Padre Cac...|Porto Alegre, Rio...|Brazil|     50128|     grass|https://media-3.a...|
|       218|Estádio Nilton Sa...|Rua Arquias Corde...|      Rio de Janeiro|Brazil|     46931|     grass|https://media-1.a...|
|       258|      Allianz Parque|Rua Turiaçu 1840,...|São Paulo, São Paulo|Brazil|     43713|     grass|https://media-

### Inserindo data de ingestão

In [27]:
# Função para adição de data de ingestão
df = adicionar_data_de_ingestao(
    df = df,
    nome_coluna = 'data_de_ingestao',
    incluir_hora = False,
    add_tempo = -3,
    unidade_de_medida = 'horas'
)

df.show()

+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+----------------+
|id_estadio|          nm_estadio|            endereço|              cidade|  pais|capacidade|superficie|         link_imagem|data_de_ingestao|
+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+----------------+
|       204|Estadio Jornalist...|Rua Professor Eur...|Rio de Janeiro, R...|Brazil|     78838|     grass|https://media-2.a...|      2024-05-01|
|       216|    Arena Fonte Nova|Rua Lions Club, N...|     Salvador, Bahia|Brazil|     56500|     grass|https://media-1.a...|      2024-05-01|
|       244|Estádio José Pinh...|Avenida Padre Cac...|Porto Alegre, Rio...|Brazil|     50128|     grass|https://media-3.a...|      2024-05-01|
|       218|Estádio Nilton Sa...|Rua Arquias Corde...|      Rio de Janeiro|Brazil|     46931|     grass|https://media-1.a...|      2024-05-01|

In [28]:
df.printSchema()

root
 |-- id_estadio: string (nullable = true)
 |-- nm_estadio: string (nullable = true)
 |-- endereço: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- capacidade: integer (nullable = true)
 |-- superficie: string (nullable = true)
 |-- link_imagem: string (nullable = true)
 |-- data_de_ingestao: date (nullable = true)



### Inserindo o arquivo tratado na camada silver

In [23]:
df.write.mode('overwrite').parquet(f"{caminho_silver}/{arquivo_fonte}")

In [24]:
dbutils.notebook.exit(f'Arquivo {arquivo_fonte} foi processado e carregado na camada silver.')